In [2]:
import numpy as np
import pandas as pd
from utils import *
from scipy.sparse import csr_matrix, vstack, hstack

In [3]:
SEED_VAL = 200;
WORK_DIR = os.getcwd();
data_subset = "_10Percent"
YELP_DATA_CSV_DIR = os.path.join(WORK_DIR, "data", "csv")
YELP_DATA_SPARSE_MATRIX_DIR = os.path.join(WORK_DIR, "data", "sparse_matrix")
YELP_DATA_FEAT_ADD = os.path.join(WORK_DIR, "data", "pos_neg")
YELP_DATA_SPARSE_MATRIX_DIR_FEAT_ADD = os.path.join(WORK_DIR, "data", "sparse_matrix_feat_add")

In [4]:
make_sure_path_exists(YELP_DATA_SPARSE_MATRIX_DIR)

In [5]:
read_filename = os.path.join(YELP_DATA_CSV_DIR, 'business_review_user' + data_subset + '.csv')
df_data = pd.read_csv(read_filename, engine='c', encoding='utf-8')

C:\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [6]:
df_data_preprocessed_review = df_data.copy();
%time df_data_preprocessed_review['review_text'] = df_data_preprocessed_review['review_text'].apply(lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem_and_restring)

Wall time: 5min 53s


In [7]:
df_data.review_text[1]

u"Great outdoor patio dining area. Great happy hour. Great service.\r\n\r\nOutdoor patio dining has a beautiful mesquite tree for ambiance and blocking out the sun while the center fireplace keeps you warm. \r\n\r\nWe had:\r\nQueso Skillet with warm tortilla chips - amazing!\r\nMac N Cheese with Chorizo - fabulous! one of the best mac n cheeses I've ever had!\r\nCarne Asada on a Potato Pancake - was ok. Sounded excellent, tasted decent.\r\n\r\nFriendly and good food. But the ambiance really puts it over the top as a great dining experience. I'd be back with a group of friends to lounge, play cornsack or bocce ball during happy hour."

In [8]:
df_data_preprocessed_review.review_text[1]

u'great outdoor patio dine area great happi hour great servic outdoor patio dine beauti mesquit tree ambianc block sun center fireplac keep warm queso skillet warm tortilla chip amaz mac n chees chorizo fabul one best mac n chees ive ever carn asada potato pancak ok sound excel tast decent friendli good food ambianc realli put top great dine experi id back group friend loung play cornsack bocc ball happi hour'

In [9]:
df_data['pos_words'] = ''
df_data['neg_words'] = ''
df_data['net_senti'] = ''
df_data['review_senti'] = ''

In [10]:
POS_WORDS_FILE = os.path.join(YELP_DATA_FEAT_ADD, 'positive-words.txt')
NEG_WORDS_FILE = os.path.join(YELP_DATA_FEAT_ADD, 'negative-words.txt')

In [11]:
with open(POS_WORDS_FILE) as f:
    pos_words1 = f.read().split()[213:]

with open(NEG_WORDS_FILE) as f:
    neg_words1 = f.read().split()[213:]    
    
print "First 5 positive words %s " % pos_words1[:5]
print "First 5 negative words %s" % neg_words1[:5]

print "Number of positive words %d" % len(pos_words1)

print "Number of negative words %d" % len(neg_words1)

all_words_with_sentiment = pos_words1 + neg_words1

print "Total number of words %d" % len(all_words_with_sentiment)

First 5 positive words ['a+', 'abound', 'abounds', 'abundance', 'abundant'] 
First 5 negative words ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable']
Number of positive words 2006
Number of negative words 4783
Total number of words 6789


In [12]:
def lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords(s):
    s = remove_numbers_in_string(s)
    s = lowercase_remove_punctuation(s)
    s = remove_stopwords(s)
    token_list = []
    token_list.append(nltk.word_tokenize(s))
    #token_list = filter_out_more_stopwords(token_list)    

    #token_list = stem_token_list(token_list)
    return token_list

In [13]:
all_reviews = [] # Initialize an empty list of sentences for each review

print "Parsing sentences from training set"
for review in df_data["review_text"]:
    all_reviews += lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords(review)

print len(all_reviews)
print len(df_data)
print all_reviews[0]

Parsing sentences from training set
219285
219285
[u'located', u'chevron', u'gas', u'station', u'lot', u'quick', u'easy', u'done', u'less', u'minutes', u'smog', u'guy', u'pleasant', u'enough', u'end']


In [14]:
i = 0
for sentence in all_reviews:
    num_positive = sum([r in pos_words1 for r in sentence])
    num_negative = sum([r in neg_words1 for r in sentence])
    net_sentiment = num_positive - num_negative
     
#   num_positive #adds number of positive words to the dataframe for each review
#   num_negative #adds number of negative words to the dataframe for each review
#   net_sentiment #adds total weight to the dataframe for each review
    
    
    df_data.set_value(i, 'pos_words', num_positive)
    df_data.set_value(i, 'neg_words', num_negative)
    df_data.set_value(i, 'net_senti', net_sentiment)
    

    if(net_sentiment > 0):
        #adds 1 to the dataframe for each review denoting a positive tendency of the review
        df_data.set_value(i, 'review_senti', 1)
    elif(net_sentiment < 0):
        #adds -1 to the dataframe for each review denoting a negative tendency of the review
         df_data.set_value(i, 'review_senti', -1)
    else:
        #adds 0 to the dataframe for each review denoting a neutral tendency of the review
         df_data.set_value(i, 'review_senti', 0)
    i = i + 1

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [15]:
df_data[['review_text','pos_words','neg_words','net_senti','review_senti','review_stars']]

,review_text,pos_words,neg_words,net_senti,review_senti,review_stars
0,It's located in the Chevron gas station lot. Q...,3,0,3,1,4
1,Great outdoor patio dining area. Great happy h...,17,0,17,1,4
2,for the record... cabo fish taco has never bee...,0,0,0,0,5
3,Came in with a groupon but will definitely ret...,7,2,5,1,5
4,I love the Mack!\r\n\r\nI've come to the Mack ...,18,5,13,1,4
5,"This Panda Express is not the best, neither th...",3,0,3,1,1
6,always do a good job. friendly service. meti...,5,0,5,1,4
7,Excellent baby back ribs. the meat is tender a...,3,2,1,1,4
8,Let me start by saying that I have been readin...,23,14,9,1,2
9,Was there last night and my Co - workers were ...,2,3,-1,-1,1


In [16]:
vectorizer = CountVectorizer(analyzer = "word",
                                 tokenizer = None,
                                 preprocessor = None,
                                 ngram_range = (1, 1),
                                 strip_accents = 'unicode')

In [17]:
feature_matrix = vectorizer.fit_transform(df_data_preprocessed_review.review_text)

In [18]:
feature_matrix

<219285x155897 sparse matrix of type '<type 'numpy.int64'>'
	with 10712193 stored elements in Compressed Sparse Row format>

In [19]:
a = csr_matrix(df_data.pos_words,dtype=int)
a = csr_matrix.transpose(a)


b = csr_matrix(df_data.neg_words,dtype=int)
b = csr_matrix.transpose(b)


c = csr_matrix(df_data.net_senti,dtype=int)
c = csr_matrix.transpose(c)


d = csr_matrix(df_data.review_senti,dtype=int)
d = csr_matrix.transpose(d)


In [20]:
feature_matrix = csr_matrix(hstack((feature_matrix,a)))
feature_matrix = csr_matrix(hstack((feature_matrix,b)))
feature_matrix = csr_matrix(hstack((feature_matrix,c)))
feature_matrix = csr_matrix(hstack((feature_matrix,d)))

In [21]:
feature_matrix

<219285x155901 sparse matrix of type '<type 'numpy.int64'>'
	with 11487249 stored elements in Compressed Sparse Row format>

In [22]:
spare_matrix_file = os.path.join(YELP_DATA_SPARSE_MATRIX_DIR_FEAT_ADD, "bagWords_feat_add"+ data_subset)
save_sparse_csr(spare_matrix_file, feature_matrix)

In [23]:
test = load_sparse_csr(spare_matrix_file + ".npz")

In [24]:
print np.array_equal(feature_matrix.data, test.data)
print np.array_equal(feature_matrix.indices, test.indices)
print np.array_equal(feature_matrix.indptr, test.indptr)
print np.array_equal(feature_matrix.shape, test.shape)

True
True
True
True
